In [1]:
import pandas as pd
import os
import shutil
import subprocess

In [2]:
import pandas as pd

file_path = 'PN09-1_data.txt'

# Carregar a primeira linha como cabeçalho
df_header = pd.read_csv(file_path, nrows=1)

# Definir o intervalo de linhas a serem lidas
start_row = 0  # Linha inicial (após o cabeçalho, 0-indexed)
end_row = 3752446 + 200000    # Linha final (linha a ser lida, inclusive)

# Carregar o intervalo específico de linhas
df = pd.read_csv(file_path, skiprows=range(1, start_row), nrows=end_row - start_row + 1, header=None)

# Definir as colunas do DataFrame com base no cabeçalho
df.columns = df_header.columns

print(df)

C:\Users\orlando\AppData\Local\Temp\ipykernel_34592\3653068342.py:13: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, skiprows=range(1, start_row), nrows=end_row - start_row + 1, header=None)


                Time       1       2       3       4       5       6       7  \
0               Time   1.000   2.000   3.000   4.000   5.000   6.000   7.000   
1           0.000000  23.625  46.500  39.000  48.375  32.875  30.125  44.625   
2           0.001953  23.750  46.000  35.500  40.500  38.250  30.375  42.875   
3           0.003906  24.875  46.500  38.875  54.500  28.000  30.750  44.375   
4           0.005859  24.500  48.875  47.375  75.500  23.875  30.875  51.000   
...              ...     ...     ...     ...     ...     ...     ...     ...   
3952442  7719.611328  15.000  11.000  -5.625   6.625  -3.375  -1.750   3.375   
3952443  7719.613281  15.625   9.125  -8.500   5.500  -6.250  -3.875   9.750   
3952444  7719.615234  13.875   8.375  -9.500   5.375  -6.250  -4.750   2.375   
3952445  7719.617188  10.875   7.750  -8.375   7.500  -1.625  -6.000   6.250   
3952446  7719.619141  10.625   7.625  -7.625   6.750   0.500  -5.000   9.125   

              8       9  ...       36  

General names
- 09_Fp1_General  BLACK
- 09_O1_General   RED
- 09_Cz_General   GREEN
- 09_Fp2_General  BLUE
- 09_O2_General   Cyan

Pre-ictal names
- 09_Fp1_Pre-ictal  BLACK
- 09_O1_Pre-ictal   RED
- 09_Cz_Pre-ictal   GREEN
- 09_Fp2_Pre-ictal  BLUE
- 09_O2_Pre-ictal   Cyan

Ictal names
- 09_Fp1_Ictal      BLACK
- 09_O1_Ictal       RED
- 09_Cz_Ictal       GREEN
- 09_Fp2_Ictal      BLUE
- 09_O2_Ictal       Cyan

Post-ictal names
- 09_Fp1_Post-ictal  BLACK
- 09_O1_Post-ictal   RED
- 09_Cz_Post-ictal   GREEN
- 09_Fp2_Post-ictal  BLUE
- 09_O2_Post-ictal   Cyan

In [3]:
channels_columns = [
    ['1', 'C1'],   # B -> channel 1,  Fp1 Black
    ['5', 'C5'],   # F -> channel 5,  O1 Red
    ['15', 'C15'], # P -> channel 15, Cz Green
    ['17', 'C17'], # R -> channel 17, Fp2 Blue
    ['21', 'C21']  # V -> channel 21  O2 Cyan
]

In [4]:
def time_to_seconds(time_str):
    h, m, s = map(int, time_str.split('.'))
    return h * 3600 + m * 60 + s

In [5]:
# Seizure n 1:
# File name: PN09-1.edf
# Registration start time: 14.08.54
# Registration end time: 16.26.07
# Seizure start time: 16.09.43 
# Seizure end time: 16.11.03 

registration_start_time_sec = time_to_seconds('14.08.54')
seizure_start_time_sec = time_to_seconds('16.09.43')
seizure_end_time_sec = time_to_seconds('16.11.03 ')

start_line_ictal = (seizure_start_time_sec - registration_start_time_sec) * 512 - 2 # Taking the first two lines out of the count
end_line_ictal =   (seizure_end_time_sec - registration_start_time_sec) * 512   - 2 # Taking the first two lines out of the count

# Show the results
print(f"Start Line Ictal: {start_line_ictal}")
print(f"End Line Ictal: {end_line_ictal}")

Start Line Ictal: 3711486
End Line Ictal: 3752446


## Cutting the ICTAL series

In [6]:
result_folder = 'ICTAL'
if os.path.exists(result_folder):
    shutil.rmtree(result_folder)
os.makedirs(result_folder)

for column, channel in channels_columns:
    current_section = df.loc[start_line_ictal:end_line_ictal, column]
    values = current_section.values
    
    final_file_path = os.path.join(result_folder, channel + '.txt')
    with open(final_file_path, 'w') as f:
        for value in values:
            f.write(f"{value}\n")

## Cutting the GENERAL series

In [7]:
start_line_general = (start_line_ictal + end_line_ictal)/2 - 76800 # Calculating the general starting point based on 0.2.30 (2 minutes and 30 seconds) before the ictal starts
end_line_general = (start_line_ictal + end_line_ictal)/2 + 76800   # Calculating the general ending point based on 0.2.30 (2 minutes and 30 seconds) after the ictal ends

result_folder = 'GENERAL'
if os.path.exists(result_folder):
    shutil.rmtree(result_folder)
os.makedirs(result_folder)

for column, channel in channels_columns:
    current_section = df.loc[start_line_general:end_line_general, column]
    values = current_section.values
    
    final_file_path = os.path.join(result_folder, channel + '.txt')
    with open(final_file_path, 'w') as f:
        for value in values:
            f.write(f"{value}\n")

## Cutting the PRE ICTAL series

In [8]:
start_line_pre_ictal = start_line_ictal - (abs(start_line_ictal - end_line_ictal))  # Calculating the start of the pre-ictal based on the start of the ictal
end_line_pre_ictal = start_line_ictal 

result_folder = 'PRE_ICTAL'
if os.path.exists(result_folder):
    shutil.rmtree(result_folder)
os.makedirs(result_folder)

for column, channel in channels_columns:
    current_section = df.loc[start_line_pre_ictal:end_line_pre_ictal, column]
    values = current_section.values
    
    final_file_path = os.path.join(result_folder, channel + '.txt')
    with open(final_file_path, 'w') as f:
        for value in values:
            f.write(f"{value}\n")

## Cutting the POST ICTAL series

In [9]:
start_line_post_ictal = end_line_ictal 
end_line_post_ictal = end_line_ictal + (abs(start_line_ictal - end_line_ictal)) # Calculating the end of the post-ictal based on the end of the ictal

result_folder = 'POST_ICTAL'
if os.path.exists(result_folder):
    shutil.rmtree(result_folder)
os.makedirs(result_folder)

for column, channel in channels_columns:
    current_section = df.loc[start_line_post_ictal:end_line_post_ictal, column]
    values = current_section.values
    
    final_file_path = os.path.join(result_folder, channel + '.txt')
    with open(final_file_path, 'w') as f:
        for value in values:
            f.write(f"{value}\n")

# Running DFA

In [10]:
def run_dfa_exe(directory, input_files, output_files):
    dfa_exe_path = 'dfa12.exe'
    
    command = [dfa_exe_path]
    
    input_str = (
        f"{input_files[0]}\n"
        f"{input_files[1]}\n"
        f"{output_files[0]}\n"
        f"{output_files[1]}\n"
    )
    
    process = subprocess.Popen(command, stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    stdout, stderr = process.communicate(input=input_str)
    
    if process.returncode != 0:
        print(f"Erro ao executar dfa12.exe na pasta {directory}: {stderr}")
    else:
        print(f"Saída do dfa12.exe na pasta {directory}:\n{stdout}")

def process_directory(directory):
    files = [f for f in os.listdir(directory) if f.endswith('.txt')]
    files.sort() 

    for i in range(0, len(files), 2):
        if i + 1 < len(files):
            input_files = [os.path.join(directory, files[i]), os.path.join(directory, files[i + 1])]
            output_files = [os.path.join(directory, files[i].replace('.txt', '.dat')), os.path.join(directory, files[i + 1].replace('.txt', '.dat'))]

            print("input_files a:", input_files)
            print("output_files a:", output_files)
            run_dfa_exe(directory, input_files, output_files)
        else:
            input_files = [os.path.join(directory, files[i]), os.path.join(directory, files[i])]
            output_files = [os.path.join(directory, files[i].replace('.txt', '.dat'))] * 2

            print("input_files b:", input_files)
            print("output_files b:", output_files)
            run_dfa_exe(directory, input_files, output_files)

directories = ['GENERAL', 'ICTAL', 'PRE_ICTAL', 'POST_ICTAL']

for directory in directories:
    if os.path.exists(directory):
        process_directory(directory)
    else:
        print(f"Pasta {directory} não encontrada.")


input_files a: ['GENERAL\\C1.txt', 'GENERAL\\C15.txt']
output_files a: ['GENERAL\\C1.dat', 'GENERAL\\C15.dat']
Saída do dfa12.exe na pasta GENERAL:
Entre com o nome da primeira serie a ser lida:
Entre com o nome da segunda serie a ser lida:
(....com o mesmo numero de pontos N):
Qual o nome do primeiro arquivo de saida (para o DFA1)?
Qual o nome do segundo arquivo de saida (para o DFA2)?

input_files a: ['GENERAL\\C17.txt', 'GENERAL\\C21.txt']
output_files a: ['GENERAL\\C17.dat', 'GENERAL\\C21.dat']
Saída do dfa12.exe na pasta GENERAL:
Entre com o nome da primeira serie a ser lida:
Entre com o nome da segunda serie a ser lida:
(....com o mesmo numero de pontos N):
Qual o nome do primeiro arquivo de saida (para o DFA1)?
Qual o nome do segundo arquivo de saida (para o DFA2)?

input_files b: ['GENERAL\\C5.txt', 'GENERAL\\C5.txt']
output_files b: ['GENERAL\\C5.dat', 'GENERAL\\C5.dat']
Saída do dfa12.exe na pasta GENERAL:
Entre com o nome da primeira serie a ser lida:
Entre com o nome da seg